In [ ]:
# 导入必要的库
import RPi.GPIO as GPIO
import time
import string
import serial


In [ ]:
#设置GPIO口为BCM编码方式
GPIO.setmode(GPIO.BCM)

#忽略警告信息
GPIO.setwarnings(False)

In [ ]:
#初始化串口，如果因权限问题，请打开系统终端，运行命令获取权限
# sudo chmod 777 /dev/ttyTHS1
ser = serial.Serial("/dev/ttyTHS1", 115200, timeout = .01)
#print("serial.isOpen()")

In [ ]:
#控制舵机，index为舵机的ID号，value为舵机的位置，s_time为舵机的运行时间
def Servo_control(index, value, s_time):
    pack1 = 0xff
    pack2 = 0xff
    id = index & 0xff
    len = 0x07
    cmd = 0x03
    addr = 0x2A
    pos1 = (value >> 8) & 0x00ff
    pos2 =  value & 0x00ff 
    time1 = (s_time >> 8) & 0x00ff
    time2 = s_time & 0x00ff
    checknum = (~(id + len + cmd + addr + pos1 + pos2 + time1 + time2)) & 0xff

    data = [pack1, pack2, id, len, cmd, addr, pos1, pos2, time1, time2, checknum]
#     print(bytes(data))
    ser.write(bytes(data))


In [ ]:
# 设置舵机ID号
def Servo_Set_ID(index):
    if index < 1 or index > 250:
        return None
    
    pack1 = 0xff
    pack2 = 0xff
    id = 0xfe
    len = 0x04
    cmd = 0x03
    addr = 0x05
    set_id = index & 0xff

    checknum = (~(id + len + cmd + addr + set_id)) & 0xff

    data = [pack1, pack2, id, len, cmd, addr, set_id, checknum]
#     print(bytes(data))
    ser.write(bytes(data))

In [ ]:
def main():
    index = 0x01
    
    Servo_Set_ID(index)
    time.sleep(.1)
    
    
    while True:
        Servo_control(index, 3100, 1000)
        time.sleep(2)
        Servo_control(index, 900, 1000)
        time.sleep(2)

In [ ]:
try:
    main()
except KeyboardInterrupt:
    pass

ser.close()
GPIO.cleanup()